In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.optimize import curve_fit
from scipy.integrate import cumulative_trapezoid
from scipy.interpolate import CubicSpline
from scipy.interpolate import RegularGridInterpolator

In [ ]:
class IsingModel:
    def __init__(self, L, T):
        #parâmetros do modelo
        self.L = L
        self.T = T

        #array com os spins
        self.spins = np.ones((L,L))

        #Inicializar energia total e magnet total
        self.energy_total = -2*L*L
        self.magnet_total = L*L

        #arrays para guardar evoluções das variáveis
        self.energies= [-2*L*L]
        self.magnetizations= [L*L]

    def calc_ener_spin(self, i, j):
        #contribuição do spin ij para a energia total
        spin= self.spins[i,j]
        neighbours= (
            self.spins[(i+1)%self.L, j] + 
            self.spins[i, (j+1)%self.L] + 
            self.spins[(i-1)%self.L, j] + 
            self.spins[i, (j-1)%self.L])
        energy=-spin*neighbours
        return energy

    def calc_ener_total( self ):
        energy = 0
        for i in range(self.L):
            for j in range(self.L):
                energy+= -self.spins[i,j]*(self.spins[(i+1)%self.L,j]+ self.spins[i,(j+1)%self.L])
        return energy
    
    def calc_ener ( self ):
        #calcular a energia por spin do sistema
        return self.energy_total/(self.L*self.L)
    
    def calc_mag( self ):
        #calcular a magnetização por spin do sistema
        return self.magnet_total/(self.L*self.L)
    

    def iter_monte_carlo(self, n_iter):
        L = self.L
        for _ in range(n_iter):
            i, j = np.random.randint(0, L, 2)
            delta_E = -2*self.calc_ener_spin(i, j)

            #Se delta_E < 0, exp(-delta_E/(self.T)) > 1 => proposta aceite
            if np.random.rand() < np.exp(-delta_E/(self.T)):
                self.spins[i, j] *= -1
                self.energy_total += delta_E

                #colocamos um + pois já trocámos o sinal do spin
                self.magnet_total += 2*self.spins[i, j]
            
            #atualizar as evoluções de E e Magnet
            self.energies.append(self.energy_total)
            self.magnetizations.append(self.magnet_total)
    
    @property
    def energy(self):
        #para aceder ao array com as energias
        return np.array(self.energies)
    
    @property
    def magnetization(self):
        #para aceder ao array com as magnetizações
        return np.array(self.magnetizations)

## 1. Tempo de Termalização

**a)** Para cada combinação de L e T, corremos a simulação iter_monte_carlo, obtendo uma lista de como variou a energia a cada passo da iteração (`ising.energies`). A n-ésima entrada dessa lista (`ising.energies[n-1]`) dá a energia total do sistema após n iterações, pelo que a energia média, em função de n, é `ising.energies[n-1]/L^2`.

In [ ]:
L_array = np.array([16, 32, 64, 128])
T_array = np.array([1, 2, 3, 4])
N_iter = 300_000

fig, axes = plt.subplots(len(L_array), len(T_array), figsize=(20, 15))
fig.suptitle('Energia média em função do nº de Iterações', fontsize=20)

for i, L in enumerate(L_array):
    for j, T in enumerate(T_array):
        
        ising = IsingModel(L, T)
        ising.iter_monte_carlo(N_iter)
        avg_ener = np.array(ising.energies[1:])/(L*L)

        ax = axes[i, j]
        ax.scatter(np.arange(1, N_iter + 1), avg_ener, s=10)
        ax.set_title(f'L = {L}, T = {T}')
        ax.set_xlabel('Iterações')
        ax.set_ylabel('Energia Média')
        ax.grid(True)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

**b)** Com o módulo scipy.optimize, ajustámos aos dados anteriores uma função como a descrita no enunciado, determinando os parâmetros `ef` e $\tau$, que são depois guardados no dicionário `fit_results`. Relativamente ao initial guess dado para os parâmetros: o ef foi escolhido como a média dos últimos 10% de valores; para escolher o tau, começámos por inverter a equação (4) do enunciado, para obter:

$$
\tau_{term} = - \frac{N}{\log\left( \frac{e(N)-e_f}{e_0-e_f} \right)}
$$

Para evitar erros, temos de garantir que $e_0-e_f \neq 0$ e que o argumento do logaritmo é estritamente maior que 0 e diferente de 1 (no código assinalamos essas condições com `#(1)`). Nos casos em que obtemos um erro do primeiro tipo tomamos como guess para tau um valor muito alto, para compensar pelo facto de os finais 10% de valores terem média igual à mínima energia. Fazemos o mesmo para um erro do terceiro tipo. No final fazemos a média dos taus obtidos (caso haja algum elemento determinado sem os erros que descrevemos).

In [ ]:
def ener_func(N, ef, tau):
    return ef + (e0 - ef) * np.exp(-N / tau)

def guesses_for_ef_and_tau(energy_vals, n_points):

    #o guess para ef será a média dos últimos 2% de valores
    ef_guess = np.mean(avg_ener[-int(0.02 * len(avg_ener)):])

    #o guess para tau será resultante de inverter a equação, usando ef_guess
    e0 = energy_vals[0]

    if e0 - ef_guess == 0: #(1)
        return [ef_guess, 10**5]
    
    indices = np.random.choice(np.arange(1, len(energy_vals)), size=n_points, replace=False)
    taus = []
    for i in indices:
        if energy_vals[i] < ef_guess and energy_vals[i]-ef_guess != e0 - ef_guess: #(1)
            tau = -i / np.log((energy_vals[i]-ef_guess)/ (e0 - ef_guess))
            taus.append(tau)

    if taus: #caso a lista seja não vazia
        return [ef_guess, np.mean(taus)]
    else:
        return [ef_guess, 10**5]
    

# Dicionário para guardar os parâmetros de ajuste
fit_results = {}

for i, L in enumerate(L_array):
    for j, T in enumerate(T_array):
        
        ising = IsingModel(L, T)
        ising.iter_monte_carlo(N_iter)
        N_values = np.arange(0, N_iter + 1)
        avg_ener = np.array(ising.energies) / (L * L)
        e0 = avg_ener[0]

        # Chamada do ajuste com estimativas iniciais
        parameters, _ = curve_fit(ener_func, N_values, avg_ener, p0=guesses_for_ef_and_tau(avg_ener, 10))
        ef, tau = parameters

        # A chave (L,T) estará associada a um dicionário onde se guardam os valores ef e tau
        fit_results[(L, T)] = {'ef': ef, 'tau': min(100000, tau)}

print("Resultados do ajuste (ef e tau):\n")
for (L, T), params in fit_results.items():
    ef = params['ef']
    tau = params['tau']
    print(f"L = {L}, T = {T}: ef = {ef:.5f}, tau = {tau:.2f}")

**c)** Representação Gráfica da função tau_term, com os resultados das alíneas anteriores. Para a representação gráfica de tau(L, T), optámos por duas formas distintas: uma delas através do plot de 4 gráficos distintos de "tempo de termalização em função da temperatura", um para cada L, e um plot da função interpolada à grid L T (excluindo a temperatura T=1 para facilitar a comparação entre diferentes zonas). Relativamente à análise destes dados, podemos dizer que, para temperaturas a partir de T=2, se verifica que o tempo de termalização aumenta com o L. Para temperaturas mais baixas, a equação (4) não fornece claramente um modelo adequado, pelo que a significância dos valores obtidos para tau não é muita (daí também termos excluído T=1 no segundo plot).

In [ ]:
tau_array = np.array([[fit_results[l, t]['tau']  for t in T_array] for l in L_array])

plt.figure(figsize=(10, 5))

for i in range(len(L_array)):
    plt.plot(T_array, tau_array[i,], label=f'L = {L_array[i]}')

plt.title(f'τ_term(L, T)')
plt.yscale('log')
plt.xlabel('Temperatura')
plt.ylabel('Tempo de Termalização')
plt.legend()
plt.grid()
plt.show() 


interp_tau = RegularGridInterpolator((L_array, T_array), tau_array, bounds_error = False, method='cubic')

#Corrected Tau é apenas para visualização gráfico
corrected_tau_array = np.array([[fit_results[l, t]['tau']  for t in T_array if t != 1] for l in L_array])
corrected_interp_tau = RegularGridInterpolator((L_array, T_array[-3:]), corrected_tau_array, method='linear')

# Representação gráfica (mapa de calor ou contorno)
L_grid, T_grid = np.meshgrid(L_array, T_array[-3:], indexing='ij')

plt.figure(figsize=(8, 6))
cp = plt.contourf(T_grid, L_grid, corrected_tau_array, levels=20, cmap='plasma')
plt.colorbar(cp, label=r'$\tau_{\mathrm{term}}(L,T)$')
plt.xlabel('Temperatura T')
plt.ylabel('Tamanho L')
plt.title(r'Mapa de $\tau_{\mathrm{term}}(L, T)$ com interpolação')
plt.grid(True)
plt.tight_layout()
plt.show()


## 2. Determinação da Temperatura Crítica

**a)** Estudo, para cada L, da energia e magnetização médias (por spin) em função da temperatura.

In [ ]:
h = 0.2
T_values = np.arange(1, 4.1, h)
N_measurements = 1_000_000

# Iremos armazenar os resultados no seguinte dicionário:
results = {L: {'e_mean': [], 'e2_mean': [], 'e_std': [], 'm_mean': [], 'm_std': []} for L in L_array}

fig, axes = plt.subplots(4, 2, figsize=(12, 16))
fig.suptitle('Energia média e Magnetização média vs Temperatura para diferentes L', fontsize=20)

for i, L in enumerate(L_array):
    for T in tqdm(T_values, desc=f"L={L}"):
        ising = IsingModel(L, T)
        if T>=2.0:
            N_term = int(4*corrected_interp_tau((L,T)))
        else:
            N_term = int(4*interp_tau((L,T)))
        ising.iter_monte_carlo(N_term + N_measurements)

        e_data = np.array(ising.energies[-N_measurements:]) / (L*L)
        m_data = np.abs(np.array(ising.magnetizations[-N_measurements:])) / (L*L)

        # Médias e desvios padrão
        mean_e = np.mean(e_data)
        mean_e2 = np.mean(e_data**2)
        e_std = np.std(e_data, ddof=0)

        mean_m = np.mean(m_data)
        mean_m2 = np.mean(m_data**2)
        m_std = np.std(m_data, ddof=0)
        
        results[L]['e_mean'].append(mean_e)
        results[L]['e2_mean'].append(mean_e2)
        results[L]['e_std'].append(e_std)
        results[L]['m_mean'].append(mean_m)
        results[L]['m_std'].append(m_std)

    #Para realizar o plot
    ax_e = axes[i, 0]
    ax_e.errorbar(T_values, results[L]['e_mean'], yerr=results[L]['e_std'], fmt='o', capsize=3, label=f'L={L}')
    ax_e.set_xlabel('Temperatura T')
    ax_e.set_ylabel('Energia média e')
    ax_e.set_title(f'Energia média (L={L})')
    ax_e.grid(True)
    ax_m = axes[i, 1]
    ax_m.errorbar(T_values, results[L]['m_mean'], yerr=results[L]['m_std'], fmt='o', capsize=3, color='orange', label=f'L={L}')
    ax_m.set_xlabel('Temperatura T')
    ax_m.set_ylabel('Magnetização média m')
    ax_m.set_title(f'Magnetização média (L={L})')
    ax_m.grid(True)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

**b)** Partimos da equação $c = \frac{\partial e}{\partial T} $ para determinar a capacidade calorífica em função da temperatura para cada L. Fazemos isto a partir dos arrays `T_values` e `results[L]['e_mean']`, calculado na secção anterior, e do método da diferença central para o cálculo da derivada nos pontos interiores do array. Para estes, o erro é dado por $\frac{\sqrt{(\sigma_{i+1}^{e})^2+(\sigma_{i-1}^{e})^2}}{2h} $ em que $\sigma_{i+1}^{e}$ é o erro (em particular o desvio padrão determinado na alínea anterior e guardado em `results[L]['e_std']`) associado à `e_mean`. Para os extremos do array, usamos o método da diferença à direita (extremo esquerdo) e à esquerda (extremo direito), tomando como erro uma expressão análoga, em que os $\sigma_e$ considerados são relativos aos pontos que entram na subtração e o denominador da expressão é apenas $h$. Isto é realizado com o auxílio de uma função auxiliar `discrete_derivative_for_data`.

Obtivémos também o valor de T que maximizava c, a Tc. Para todos os valores de L (se considerados pontos suficientes - $10^6$ como número de medições é suficiente) temos que o valor é 2.4. Isto significa que é bastante provável que a temperatura crítica esteja entre 2.3 e 2.5. Para uma análise mais detalhada, será obtido um ajuste da curva que modela esta quantidade (alínea 2d)

In [ ]:
c_values_method_1 = {L: {'c': [], 'c_err': []} for L in L_array}

def discrete_derivative_for_data(T, Y, Y_err):
    if len(T) != len(Y):
        raise ValueError("T and Y must have the same length")
    dY_values = [(Y[1]-Y[0])/(T[1]-T[0])]
    Err_values = [np.sqrt(Y_err[0]*Y_err[0] + Y_err[1]*Y_err[1])/(T[1]-T[0])]
    for i in range(1, len(T)-1):
        dY_values.append((Y[i+1]-Y[i-1])/(T[i+1]-T[i-1]))
        Err_values.append(np.sqrt(Y_err[i-1]*Y_err[i-1] + Y_err[i+1]*Y_err[i+1])/(T[i+1]-T[i-1]))
    dY_values.append((Y[-1]-Y[-2])/(T[-1]-T[-2]))
    Err_values.append(np.sqrt(Y_err[-1]*Y_err[-1] + Y_err[-2]*Y_err[-2])/(T[-1]-T[-2]))
    return dY_values, Err_values

for L in L_array:
    c_values, c_error = discrete_derivative_for_data(T_values, results[L]['e_mean'], results[L]['e_std'])
    c_values_method_1[L]['c'] = c_values
    c_values_method_1[L]['c_err'] = c_error
    idx_max = np.argmax(c_values_method_1[L]['c'])
    Tc = T_values[idx_max]
    c, Error = c_values_method_1[L]['c'][idx_max], c_values_method_1[L]['c_err'][idx_max]
    print(f"L = {L}: Tc = {Tc:.1f} e c = {c:.6f} +/- {Error:.6f}")


# Plot da capacidade calorífica em função de T, com barras de erro

plt.figure(figsize=(10, 6))

for L in L_array:
    plt.errorbar(T_values, c_values_method_1[L]['c'], yerr=c_values_method_1[L]['c_err'], fmt='o', capsize=3, label=f'L={L}')

plt.xlabel('Temperatura T')
plt.ylabel('Capacidade Calorífica c')
plt.title('Capacidade Calorífica vs Temperatura (Método 1)')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


**c)** Capacidade calorífica calculada a partir da base estatística do método de Metropolis-Hastings. Nota Importante sobre a fórmula utilizada: uma vez que as `energies` já calculadas são na verdade as energias médias por spin, temos que: 

$$
\frac{\langle E^2 \rangle- \langle E \rangle^2}{T^2 L^2} = L^2 \frac{\langle e^2 \rangle- \langle e \rangle^2}{T^2}
$$

pelo que será esta a fórmula utilizada. Através deste segundo método, obtivémos a Temperatura crítica média $T_c = 2.392$, bastante concordante com a determinada na alínea anterior.

In [ ]:
c_values_method_2 = {L: [] for L in L_array}
stat_Tcs = {}

for L in L_array:
    for i, T in enumerate(T_values):
        
        c = L*L*(results[L]['e2_mean'][i] - results[L]['e_mean'][i]**2)/(T*T)
        c_values_method_2[L].append(c)

    index_for_Tc = np.argmax(np.array(c_values_method_2[L]))
    stat_Tc = T_values[index_for_Tc]
    stat_Tcs[L] = stat_Tc
    print(f"L = {L}: Tc = {stat_Tc:.1f}")


plt.figure(figsize=(10, 6))

for L in L_array:
    plt.plot(T_values, c_values_method_2[L], 'o', label=f'L={L}')

plt.xlabel('Temperatura T')
plt.ylabel('Capacidade Calorífica c')
plt.title('Capacidade Calorífica vs Temperatura (Método 2)')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

**d)** Modelação da capacidade calorífica a uma função como a da equação (7). Começamos por fazer um ajuste aos dados obtidos, pelo primeiro método (alínea b) e depois para os do segundo. Os dados obtidos são printed após as imagens. Pelo gráfico do primeiro, conseguimos ver que os ajustes são bastante adequados, talvez à exceção do último. Notámos que este método de ajuste (`curve_fit` do `scipy.optimize`) é bastante sensível às condições iniciais. Relativamente aos valores obtidos no último, consideramos que os valores obtidos para A e B para L = 64 e L = 128 não são muito significativos (por análise do gráfico).

In [ ]:
def c_model(T, A, B, Tc):
    return A * np.log(np.abs((T - Tc) / T)) + B

#para posterior análise, criamos o seguinte dicionário para guardar os parâmetros dos ajustes
fit_parameters = {'m1':{'A':[], 'B':[], 'Tc':[]}, 'm2':{'A':[], 'B':[], 'Tc':[]}}


'''Método 1: derivada (central) de e'''

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Método 1: Capacidade Calorífica e ajuste de $c(T) = A \\ln|\\frac{T - T_c}{T}| + B$', fontsize=20)

L_list = [16, 32, 64, 128]
for j, L in enumerate(L_list):
    T = np.array(T_values)
    c = np.array(c_values_method_1[L]['c'])

    parameters, _ = curve_fit(c_model, T, c, p0=[-1.0, 0.0, 2.35])
    A, B, Tc = parameters
    fit_parameters['m1']['A'].append(A)
    fit_parameters['m1']['B'].append(B)
    fit_parameters['m1']['Tc'].append(Tc)

    ax = axes[j // 2, j % 2]
    T_fine = np.linspace(min(T), max(T), 300)
    ax.errorbar(T, c, yerr=np.array(c_values_method_1[L]['c_err']), fmt='o', capsize=3, label='dados')
    ax.plot(T_fine, c_model(T_fine, *parameters), '-', label=f'c(T), A={A:.2f}, B={B:.2f}')
    ax.axvline(Tc, color='r', linestyle='--', label=f'$T_c$ ≈ {Tc:.3f}')
    ax.set_title(f'L = {L}')
    ax.set_xlabel('T')
    ax.set_ylabel('c(T)')
    ax.grid(True)
    ax.legend()

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


'''Método 2: Tirando proveito da base estatística do método de Metropolis-Hastings'''

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Método 2: Capacidade Calorífica e ajuste de $c(T) = A \\ln|\\frac{T - T_c}{T}| + B$', fontsize=20)

L_list = [16, 32, 64, 128]
for j, L in enumerate(L_list):
    T = np.array(T_values)
    c = np.array(c_values_method_2[L])

    #variámos o guess face à sensibilidade do método pedido às condições iniciais
    Tc_guess = 2.5 if L == 128 else 2.4 if L == 64 else 2.35
    parameters, _ = curve_fit(c_model, T, c, p0=[-1.0, 0.0, Tc_guess])
    A, B, Tc = parameters
    fit_parameters['m2']['A'].append(A)
    fit_parameters['m2']['B'].append(B)
    fit_parameters['m2']['Tc'].append(Tc)

    ax = axes[j // 2, j % 2]
    T_fine = np.linspace(min(T), max(T), 300)
    ax.plot(T, c, 'o', label='dados')
    ax.plot(T_fine, c_model(T_fine, *parameters), '-', label=f'c(T), A={A:.2f}, B={B:.2f}')
    ax.axvline(Tc, color='r', linestyle='--', label=f'$T_c$ ≈ {Tc:.3f}')
    ax.set_title(f'L = {L}')
    ax.set_xlabel('T')
    ax.set_ylabel('c(T)')
    ax.grid(True)
    ax.legend()

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

print(f"Método 1: A={np.mean(fit_parameters['m1']['A']):.2f}, B={np.mean(fit_parameters['m1']['B']):.2f}, Tc={np.mean(fit_parameters['m1']['Tc']):.3f}")
print(f"Método 2: A={np.mean(fit_parameters['m2']['A']):.2f}, B={np.mean(fit_parameters['m2']['B']):.2f}, Tc={np.mean(fit_parameters['m2']['Tc']):.3f}")

## 3. Determinação da Energia Livre de Helmholtz, F

**a)** Para calcular a energia livre de Helmholtz, é necessário calcular o integral

$\int_{T}^{10} \frac{E(T')}{T'^2} dT'$

Este integral pode ser calculado da forma usual, primitivando numericamente e fazendo a diferença do valor da primitiva nos limites do intervalo.

Utilizando o método `cumulative_trapezoid` da biblioteca `scipy`, calculámos a primitiva numérica para o integrando no domínio útil. Esta função dá como _output_ um array com os valores da primitiva. De seguida, calculámos o valor do integral fazendo a diferença entre os termos limite do _array_ da primitiva, isto é, a diferença entre os termos correspondente a $T' = 10$ e $T' = T$.

In [ ]:
#Parâmetros
L = 128
T_array = np.arange(0.1, 10.1, 0.1)
n_measures = 10000

# Armazena as energias correspondentes a cada T
energy_array = np.zeros_like(T_array)

for i, T in enumerate(T_array):
    
    #Definimos o número de iterações até termalização
    #T menor ou igual a 1 => E aprox constante => podemos considerar tau = 0
    if T <= 1:
        n_thermal = 0
    else:
        tau_value = interp_tau((L, T))
        if np.isnan(tau_value):
            n_thermal = 100000
        else:
            n_thermal = min(int(4 * tau_value), 100000)

    h_ising = IsingModel(L, T)
    h_ising.iter_monte_carlo(n_thermal)
    
    #Realizamos as medições
    energies = []
    for _ in range(n_measures):
        h_ising.iter_monte_carlo(1)
        energies.append(h_ising.energy_total)  
    energy_array[i] = np.mean(energies)

# Realizar a integração com cumulative_trapezoid
# (AQUI)
integrand_array = energy_array / (T_array**2)
primitive = cumulative_trapezoid(integrand_array, T_array, initial=0)
F_array = np.zeros_like(T_array)

for i, T in enumerate(T_array):
    if T >= 10:
        F_array[i] = -T*L*L*np.log(2)
    else:
        integral = primitive[-1] - primitive[i]
        F_array[i] = -T * (integral + L*L* np.log(2))

plt.figure(figsize=(10, 5))
plt.plot(T_array, F_array, color='blue')
plt.title('Energia livre de Helmholtz em função da temperatura')
plt.xlabel('T')
plt.ylabel('F(T)')
plt.grid(True)
plt.show()


**b)** Interpolámos a função usando a classe CubicSpline. Para calcular as raízes, importámos a função bisect do módulo scipy.optimize. Esta função utiliza o método da bissetriz para descobrir uma raíz num determinado intervalo dado.

In [ ]:
from scipy.optimize import bisect

#Criar interpolador
interp_F = CubicSpline(T_array, F_array)

#Calcular e mostrar raíz
null_F = bisect(interp_F, 0, 10)
print(f"Temperatura para a qual F(T) = 0: \n {null_F} \n\n")

#Mostrar o interpolador em gráfico
plt.figure(figsize=(10, 5)) 
plt.plot(T_array, interp_F(T_array), label='', color='green') 
plt.title('Interpolador da energia livre de Helmholtz')
plt.xlabel('T')
plt.ylabel('F(T)')
plt.grid()
plt.show()

**c)** A 2ª derivada de um interpolador feito utilizando o módulo scipy.interpolate pode ser facilmente calculada chamando o interpolador para o array de domínio de interesse, e especificando num segundo parâmetro o grau da derivada (neste caso 2). Ao fazer isto e desenhar o gráfico, podemos observar que à medida que a temperatura cresce, a 2ª derivada tende rapidamente para 0. Observamos também que a função oscila na região em torno da temperatura crítica. Isto deve-se ao facto de que, próximo da temperatura crítica, a capacidade calorífica apresenta um pico (ou divergência suave), refletindo a presença de uma transição de fase. A energia livre de Helmholtz, sendo relacionada com a energia interna, apresenta uma variação mais abrupta nas suas derivadas próximas de $T_c$ resultando nessas oscilações na segunda derivada. Longe da temperatura crítica, o sistema torna-se mais estável, e as derivadas mais altas suavizam-se, tendendo para zero porque a energia livre varia quase linearmente ou suavemente com a temperatura.

In [ ]:
#Definir a segunda derivada (chamando a classe com opção de 2ª derivada)
deriv2 = interp_F(T_array, 2)

#Gráfico da 2ª derivada
plt.figure(figsize=(10, 5))
plt.plot(T_array, deriv2, label='', color='red')
plt.title('2ª derivada do interpolador da energia livre de Helmholtz')
plt.xlabel('T')
plt.ylabel('')
plt.grid()
plt.show()